In [10]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import StructuredTool, Tool, BaseTool
from pydantic import BaseModel

llm = ChatOpenAI(temperature=0.1)

In [8]:
def plus(a, b):
    return float(a) + float(b)

agent = initialize_agent(
    llm=llm, 
    verbose=True,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    tools=[
        StructuredTool.from_function(
            func=plus,
            name="Sum Calculator",
            description="Use this to perform sums of two numbers. This tool take two arguments, both should be numbers."
        ), 
    ],   
)

prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"

agent.invoke(prompt)



> Entering new AgentExecutor chain...
To calculate the sum of these numbers, I can use the Sum Calculator tool. Let me do that for you.

> Finished chain.


{'input': 'Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12',
 'output': 'To calculate the sum of these numbers, I can use the Sum Calculator tool. Let me do that for you.'}

In [9]:
def plus(inputs):
    a, b = inputs.split(",")
    return float(a) + float(b)

agent = initialize_agent(
    llm=llm, 
    verbose=True,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_error=True,
    tools=[
        Tool.from_function(
            func=plus,
            name="Sum Calculator",
            description="Use this to perform sums of two numbers. Use this tool by sending a pair of number separated by comma.\nExample: 1,2"
        ), 
    ],   
)

prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"

agent.invoke(prompt)



> Entering new AgentExecutor chain...
I need to use the Sum Calculator to add up all the given numbers.
Action: Sum Calculator
Action Input: 355.39, 924.87
Observation: 1280.26
Thought:I have the sum of the first two numbers. Now I need to add the next number.
Action: Sum Calculator
Action Input: 1280.26, 721.2
Observation: 2001.46
Thought:I have the sum of the first three numbers. Now I need to add the next number.
Action: Sum Calculator
Action Input: 2001.46, 1940.29
Observation: 3941.75
Thought:I have the sum of the first four numbers. Now I need to add the next number.
Action: Sum Calculator
Action Input: 3941.75, 573.63
Observation: 4515.38
Thought:I have the sum of the first five numbers. Now I need to add the next number.
Action: Sum Calculator
Action Input: 4515.38, 65.72
Observation: 4581.1
Thought:I have the sum of the first six numbers. Now I need to add the next number.
Action: Sum Calculator
Action Input: 4581.1, 35.00
Observation: 4616.1
Thought:I have the sum of the fi

{'input': 'Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12',
 'output': 'The sum of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12 is $5273.38.'}

In [11]:
from typing import Type
from pydantic import BaseModel, Field


class CalCulatorToolArgsSchema(BaseModel):
    a: float = Field(description="The First number")
    b: float = Field(description="The second number")


class CalCulatorTool(BaseTool):
    name = "SumsCalculator"
    description = """
        Use this to perform sums of two numbers.
        The first and second arguments should be numbers.
        Only receives two arguments.
        """
    args_schema: Type[CalCulatorToolArgsSchema] = CalCulatorToolArgsSchema
    
    def _run(self, a, b):
        return a + b
    
    
agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_error=True,
    tools=[
        CalCulatorTool(),
    ],
)

prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"
agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `SumsCalculator` with `{'a': 355.39, 'b': 924.87}`


1279
Invoking: `SumsCalculator` with `{'a': 1279, 'b': 721.2}`


2000
Invoking: `SumsCalculator` with `{'a': 2000, 'b': 1940.29}`


3940
Invoking: `SumsCalculator` with `{'a': 3940, 'b': 573.63}`


4513
Invoking: `SumsCalculator` with `{'a': 4513, 'b': 65.72}`


4578
Invoking: `SumsCalculator` with `{'a': 4578, 'b': 35.0}`


4613
Invoking: `SumsCalculator` with `{'a': 4613, 'b': 552.0}`


5165
Invoking: `SumsCalculator` with `{'a': 5165, 'b': 76.16}`


5241
Invoking: `SumsCalculator` with `{'a': 5241, 'b': 29.12}`


5270The cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12 is $5270.

> Finished chain.


{'input': 'Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12',
 'output': 'The cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12 is $5270.'}

In [ ]:
from langchain.utilities import DuckDuckGoSearchAPIWrapper

class StockMarketSymbolSearchToolSchema(BaseModel):
    query: str = Field(description="The query you will search for")

class StockMarketSymbolSearchTool(BaseTool):
    name = "StockMarketSymbolSearchTool"
    description = """
        Use this tool to find the stock market symbol for a company.
        It takes a query as an arguments.
        Example query: Stock Market Symbol for Apple Company
    """
    args_schema: Type[StockMarketSymbolSearchToolSchema] = StockMarketSymbolSearchToolSchema
    
    def _run(self, query):
        ddg = DuckDuckGoSearchAPIWrapper()
        return ddg.run(query)


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_error=True,
    tools=[
        StockMarketSymbolSearchTool(),
    ],
)

prompt = "Give me information on Cloudflare's stock and help me analyze if it's a potential good investment. Also tell me what symbol does the stock has"
agent.invoke(prompt)

In [15]:
import yfinance as yf

msft = yf.Ticker("MSFT")
print("Income statement")
print(msft.income_stmt)
print()

print("Balance sheet")
print(msft.balance_sheet)

print("Cash flow statement")
print(msft.cashflow)

print("show news")
print(msft.news)

Income statement
                                                        2023-06-30  \
Tax Effect Of Unusual Items                                    0.0   
Tax Rate For Calcs                                        0.189786   
Normalized EBITDA                                   102384000000.0   
Total Unusual Items                                    -15000000.0   
Total Unusual Items Excluding Goodwill                 -15000000.0   
Net Income From Continuing Operation Net Minori...   72361000000.0   
Reconciled Depreciation                              13861000000.0   
Reconciled Cost Of Revenue                           65863000000.0   
EBITDA                                              102384000000.0   
EBIT                                                 88523000000.0   
Net Interest Income                                   1026000000.0   
Interest Expense                                      1968000000.0   
Interest Income                                       2994000000.0   
Nor

In [2]:
from langchain.agents import create_sql_agent, AgentType
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase

llm = ChatOpenAI(temperature=0.1)
db = SQLDatabase.from_uri("sqlite:///movies.sqlite")
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

agent = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

agent.invoke("Give me the 5 directors that have the highest grossing films")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: 
Thought:I can see the tables in the database. Now I need to query the schema of the relevant tables to find the columns I need for the query.
Action: sql_db_schema
Action Input: "directors, films"
Observation: Error: table_names {'films', 'directors'} not found in database
Thought:I received an error because the tables "directors" and "films" do not exist in the database. I should try querying the schema of the tables to see if they have different names or if they are not present in the database at all.
Action: sql_db_schema
Action Input: "table1, table2"
Observation: Error: table_names {'table1', 'table2'} not found in database
Thought:I received an error again because the tables "table1" and "table2" do not exist in the database. It seems that there are no tables in the database. I should inform the user that I cannot answer the question because there is no relevant data in the database.

{'input': 'Give me the 5 directors that have the highest grossing films',
 'output': "I don't know."}